# Optimización de Portafolio 28 días activo

Este código utiliza la biblioteca **PyPortfolioOpt** para construir un portafolio de inversión que maximiza el **Ratio de Sharpe** ajustado a un horizonte mensual (28 días hábiles). El proceso incluye los siguientes pasos:

In [10]:
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
import requests  # For making HTTP requests to fetch data from web pages
import os  # For interacting with the operating system (e.g., file paths)
import yfinance as yf  # For fetching financial data from Yahoo Finance

# PyPortfolioOpt library for portfolio optimization
from pypfopt.efficient_frontier import EfficientFrontier  # For creating efficient frontier and optimizing portfolios
from pypfopt import risk_models  # For calculating risk models (e.g., covariance matrix)
from pypfopt import expected_returns  # For calculating expected returns

# PyPortfolioOpt library for discrete allocation
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices  # For discrete allocation of portfolio weights

import requests  # Duplicate import, already imported above
from bs4 import BeautifulSoup  # For parsing HTML and XML documents
import pandas as pd  # Duplicate import, already imported above

from datetime import datetime  # For manipulating dates and times

# Read csv

In [11]:
# Function to read stock symbols from a file
def read_stock_symbols(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()


In [12]:

# Function to fetch adjusted close prices for given symbols
def fetch_adj_close_prices(symbols):
    adj_close_list = []
    for symbol in symbols:
        stock = yf.Ticker(symbol)
        hist = stock.history(period="max")

        # Reset index to work with the datetime as a column
        hist.reset_index(inplace=True)

        # Extract only the date, ignoring the hour
        hist['Date'] = hist['Date'].dt.date

        # Group by date and take the last closing price of each day
        daily_data = hist.groupby('Date').agg({'Close': 'last'}).rename(columns={'Close': symbol})
        
        # Append the data to the list
        adj_close_list.append(daily_data)

    # Concatenate all data along the columns axis
    return pd.concat(adj_close_list, axis=1)


In [13]:

# Function to generate all possible trading dates
def generate_all_dates(start_date, end_date):
    return pd.date_range(start=start_date, end=end_date, freq='B').date

In [14]:
current_date = datetime.now().strftime("%Y-%m-%d")

csv_file = f"{current_date}.csv"

if not os.path.exists(csv_file):
    # Read stock symbols from file
    stock_symbols = read_stock_symbols('ETFS&Stocks.txt')
    stock_symbols.insert(0, '^GSPC')  # Add S&P 500 index symbol

    # Fetch data for all stock symbols
    df = fetch_adj_close_prices(stock_symbols)

    # Get the minimum and maximum dates from the fetched data
    min_date = df.index.min()
    max_date = df.index.max()

    # Generate a complete list of business dates
    all_dates = pd.DataFrame(generate_all_dates(min_date, max_date), columns=['Date'])

    # Merge the generated dates with the fetched data, filling missing values
    df.reset_index(inplace=True)  # Reset index to make 'Date' a column
    merged_df = all_dates.merge(df, on='Date', how='left')

    # Filter out the dates where the S&P 500 index (^GSPC) has missing values
    # merged_df = merged_df[merged_df['^GSPC'].notna()]

    # Set 'Date' as index again for saving to CSV
    merged_df.set_index('Date', inplace=True)

    # Save the cleaned data to a CSV
    merged_df.to_csv(csv_file, index=True)

df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)


# Only etfs or etfs&stocks

ETF's & Stocks

In [22]:
# No se porque con estas da error
columns_to_drop = [
    "ACWI", "FAS", "GLD",
]

# Drop the columns
df.drop(columns=columns_to_drop, inplace=True)

KeyError: "['ACWI', 'FAS', 'GLD'] not found in axis"

Only Stocks

In [43]:
# # Read the column names from ETFS.txt
# with open('ETFS.txt', 'r') as file:
#     columns_to_drop = [line.strip() for line in file]

# # Assuming df is your DataFrame
# # Drop the columns
# df.drop(columns=columns_to_drop, inplace=True)

In [16]:
 # Ensure the "Results" folder exists
if not os.path.exists("Portafolios"):
    os.makedirs("Portafolios")

In [42]:
print(df.tail())

                  ^GSPC       AAXJ        ACWI        BIL       BOTZ  \
Date                                                                   
2024-09-30  5762.479980  78.449997  119.550003  91.438995  32.150002   
2024-10-01  5708.750000  79.389999  118.790001  91.459999  31.650000   
2024-10-02  5709.540039  81.239998  119.070000  91.470001  31.760000   
2024-10-03  5699.939941  80.089996  118.480003  91.489998  31.559999   
2024-10-04  5751.069824  81.050003  119.480003  91.510002  32.070000   

                   DIA        EEM        EWZ         FAS        GDX  ...  \
Date                                                                 ...   
2024-09-30  423.119995  45.860001  29.490000  129.789993  39.820000  ...   
2024-10-01  421.500000  46.189999  29.510000  127.699997  40.450001  ...   
2024-10-02  421.910004  47.099998  29.770000  127.949997  40.369999  ...   
2024-10-03  419.980011  46.540001  29.219999  126.080002  39.560001  ...   
2024-10-04  423.410004  46.970001  29.4

In [17]:
portafolio = 1000000

# Exchange rate from USD to MXN (example rate, you should use the current rate)
mxntodlls = .05
portafolio = portafolio * mxntodlls

days = 5

rf = 0.0000
rf = (1 + rf) ** (252/days) - 1

In [18]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df, frequency=days)
s = risk_models.sample_cov(df, frequency=days)

c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.

### Import Company Name

In [16]:
from openpyxl import load_workbook

def save_to_excel(file_path, expected_return, volatility, sharpe_ratio):
    
    # Load the workbook and select the active sheet
    workbook = load_workbook(file_path)
    sheet = workbook.active

    # Write the performance metrics to the specified cells
    sheet["E2"] = "Return"
    sheet["E3"] = "Volatility"
    sheet["E4"] = "Sharpe Ratio"
    sheet["F2"] = expected_return
    sheet["F3"] = volatility
    sheet["F4"] = sharpe_ratio

    # Save the workbook
    workbook.save(file_path)

# Max Sharpe Ratio

In [45]:
# Function to optimize portfolio for maximum Sharpe ratio
def optimize_for_max_sharpe(df, portfolio_value):

    ef = EfficientFrontier(mu, s)

    # Optimize for maximum Sharpe ratio
    weights = ef.max_sharpe(risk_free_rate=rf)
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

    # Get the current date
    current_date = datetime.now().strftime("%d-%m-%Y-%H-%M")

    # Save the portfolio allocation to an Excel file in the "Results" folder with the current date in the filename
    file_path = f"Portafolios/Max Sharpe {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    # Return the DataFrame
    return portfolio_df

In [49]:
portfolio_max_sharpe = optimize_for_max_sharpe(df, portfolio_value=portafolio)
print(portfolio_max_sharpe)

Expected annual return: 0.4%
Annual volatility: 1.9%
Sharpe Ratio: 0.18
   Company Ticker  Discrete Allocation
0            ABBV                   24
1     ACTINVRB.MX                  189
2            AMZN                    1
3        ASURB.MX                    1
4            AVGO                   27
5         BRKB.MX                    1
6    FIBRAPL14.MX                   10
7         GAPB.MX                    2
8       GRUMAB.MX                    6
9              HD                    3
10             MA                    2
11           NFLX                    1
12      PINFRA.MX                    8
13           Q.MX                   47
14           TSLA                    3
15        TSMN.MX                    1
16            UNH                    2
17      WALMEX.MX                   16
18            WMT                   42


c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


# Target Volatility

In [232]:
# Function to optimize portfolio for target volatility
def optimize_for_target_volatility(df, target_volatility, portfolio_value):

    ef = EfficientFrontier(mu, s)

    # Optimize for target volatility
    weights = ef.efficient_risk(target_volatility)
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

    # Get the current date
    current_date = datetime.now().strftime("%d-%m-%Y-%H-%M")

    # Save the portfolio allocation to an Excel file in the "Results" folder with the current date in the filename
    file_path = f"Portafolios/Target Volatility {target_volatility} {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    # Return the DataFrame
    return portfolio_df

In [233]:
target_volatility = 0.15

portafolio_target_volatility = optimize_for_target_volatility(df, target_volatility, portafolio)
print(portafolio_target_volatility)

Expected annual return: 0.7%
Annual volatility: 9.1%
Sharpe Ratio: 0.07
  Company Ticker  Discrete Allocation
0           TECL                  576
1           TALN                  242


# Target Return

In [234]:
# Function to optimize portfolio for target return
def optimize_for_target_return(df, target_return, portfolio_value):

    ef = EfficientFrontier(mu, s)

    # Optimize for target return
    weights = ef.efficient_return(target_return)
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

    # Get the current date
    current_date = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

    # Save the portfolio allocation to an Excel file in the "Results" folder with the current date in the filename
    file_path = f"Portafolios/Target Return {target_return} {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    # Return the DataFrame
    return portfolio_df

In [235]:
portafolio_target_return = optimize_for_target_return(df, 0.002, portafolio)
print(portafolio_target_return)

Expected annual return: 0.2%
Annual volatility: 0.8%
Sharpe Ratio: 0.19
   Company Ticker  Discrete Allocation
0             BIL                   52
1             IAU                   70
2             PSQ                  182
3             TLT                   51
4            ABBV                   12
5     ACTINVRB.MX                  150
6            AMZN                    2
7        ASURB.MX                    1
8            AVGO                   17
9     CHDRAUIB.MX                    1
10           CSCO                    2
11           FANG                    1
12    FEMSAUBD.MX                    1
13   FIBRAPL14.MX                    3
14        GAPB.MX                    1
15    GCARSOA1.MX                    2
16    GFNORTEO.MX                    1
17           GILD                    3
18          GOOGL                    6
19      GRUMAB.MX                    3
20             HD                    2
21            JNJ                    5
22             MA              

c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [236]:
portafolio_target_return = optimize_for_target_return(df, 0.001, portafolio)
print(portafolio_target_return)

Expected annual return: 0.1%
Annual volatility: 0.4%
Sharpe Ratio: 0.14
   Company Ticker  Discrete Allocation
0             BIL                  326
1             IAU                   31
2             PSQ                   84
3             TLT                   22
4            ABBV                    5
5     ACTINVRB.MX                   67
6            AMZN                    1
7            AVGO                    7
8            CSCO                    1
9     FEMSAUBD.MX                    1
10   FIBRAPL14.MX                    1
11        GAPB.MX                    1
12    GCARSOA1.MX                    1
13           GILD                    1
14          GOOGL                    3
15      GRUMAB.MX                    1
16             HD                    1
17            JNJ                    2
18             MA                    1
19           MRNA                    1
20           MSFT                    1
21      PINFRA.MX                    2
22           Q.MX              

c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [237]:
portafolio_target_return = optimize_for_target_return(df, 0.003, portafolio)
print(portafolio_target_return)

Expected annual return: 0.3%
Annual volatility: 1.3%
Sharpe Ratio: 0.19
   Company Ticker  Discrete Allocation
0             IAU                   99
1             PSQ                   49
2             TLT                   70
3            ABBV                   19
4     ACTINVRB.MX                  148
5            AMZN                    1
6        ASURB.MX                    1
7            AVGO                   23
8            FANG                    1
9    FIBRAPL14.MX                    3
10        GAPB.MX                    1
11           GILD                    2
12      GRUMAB.MX                    5
13             HD                    2
14            JNJ                    3
15             MA                    3
16           NFLX                    1
17      PINFRA.MX                    6
18           Q.MX                   32
19           TSLA                    3
20        TSMN.MX                    1
21            UNH                    2
22      WALMEX.MX              

c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


# Max Return

In [20]:
# Function to optimize portfolio for maximum return
def optimize_for_max_return(df, portfolio_value):

    ef = EfficientFrontier(mu, s)

    # Optimize for maximum return
    weights = ef.max_quadratic_utility()
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

     # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d-%H-%M")

    file_path = f"Portafolios/Max return {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    # Return the DataFrame
    return portfolio_df, leftover

In [21]:
portafolio_max_return, leftover = optimize_for_max_return(df, portfolio_value=portafolio)
print(portafolio_max_return, leftover)

ArpackNoConvergence: ARPACK error -1: ARPACK error -1: No convergence (1991 iterations, 0/1 eigenvectors converged)


        CVXPY note: This failure was encountered while trying to certify
        that a matrix is positive semi-definite (see [1] for a definition).
        In rare cases, this method fails for numerical reasons even when the matrix is
        positive semi-definite. If you know that you're in that situation, you can
        replace the matrix A by cvxpy.psd_wrap(A).

        [1] https://en.wikipedia.org/wiki/Definite_matrix
        

# Min variance

In [240]:
# Function to optimize portfolio for minimum variance
def optimize_for_min_variance(df, portfolio_value):
    ef = EfficientFrontier(mu, s)

    # Optimize for minimum variance
    weights = ef.min_volatility()
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

    # Get the current date
    current_date = datetime.now().strftime("%d-%m-%Y-%H-%M")

    # Save the portfolio allocation to an Excel file in the "Results" folder with the current date in the filename
    file_path = f"Portafolios/Min variance {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    return portfolio_df

In [241]:
portafolio_min_variance = optimize_for_min_variance(df, portfolio_value=portafolio)
print(portafolio_min_variance)

Expected annual return: -0.0%
Annual volatility: 0.1%
Sharpe Ratio: -1.29
   Company Ticker  Discrete Allocation
0            AAXJ                    1
1             BIL                  446
2             PSQ                   25
3            SQQQ                  435
4            TQQQ                   42
5             VGT                    1
6             XLV                    1
7     ACTINVRB.MX                    3
8             BMY                    1
9               F                    2
10            GCC                    5
11          GOOGL                    1
12           LCID                    3
13            LVS                    1
14     MEGACPO.MX                    1


c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
